In [2]:
from transformers import pipeline
import csv
from tqdm import tqdm

def readCSV(path): # -> # [[occ,p_she,p_he,gold_ppbs]]
    data = []
    with open(path, encoding = "UTF-8") as f:
        reader = csv.reader(f, delimiter=";")
        next(reader) # skip first line which includes the description, ie [occupation, female, male]
        data = [[x[0]] + [float(y) for y in x[1:]] for x in reader]
    return data


def probePPBS(occupations, templates, maskTag, lm):
    templates = list(map(lambda x: x.replace("____", maskTag), templates))
    unmasker = pipeline('fill-mask', model=lm) # using bert-base-multilingual-cased as model
    pred_PPBSs = {} # all occupations: {occupation: ppbs}
    for i in tqdm(range(len(occupations)), desc="Occupation"):
        occ = occupations[i]
        ppbss_occ = [] # same occupation, diff templates: [ppbs]
        for template in templates:
            setTemplate = template.replace("[OCCUPATION]", occ) #replacing [OCCUPATION] with the an occupation in the sentence
            he = 0 #init
            she = 0 # init
            results = unmasker(setTemplate) # probing

            for res in results: # for the replies returned by the language model
                token = (res["token_str"]).lower()
                #if female
                if token in {"hun", "ho", "henne", "kvinnen"}: # add more cases?
                    she += res["score"]

                #if male
                elif token in {"han", "mannen"}: # add more cases?
                    he += res["score"]

            ppbss_occ.append(round(he-she,3))

        avg = sum(ppbss_occ) / len(ppbss_occ)
        pred_PPBSs[occ] = round(avg,3)

    return pred_PPBSs # bias: + == male, - == female


def writeCSV(path, gold_data, pred_data):
    data = list(map(lambda x: [x[0],x[3],pred_data[x[0]]],gold_data)) 
    head = [["ATTRIBUTE", "GOLD", "PRED"]]

    with open(path, 'w', newline='', encoding="UTF-8") as csvfile:
        writer = csv.writer(csvfile, delimiter=";")
        writer.writerows(head)
        writer.writerows(data)

In [3]:
##########################################################################
# init data for probing

# same templates for all lm probings
# chatgpt helped with creative template ideas
templates = [
    "____ er en [OCCUPATION].",
    "____ jobber som [OCCUPATION].",
    "____ jobbet som [OCCUPATION].",
    "____ skal jobbe som [OCCUPATION].",
    "____ vil jobbe som [OCCUPATION].",
    "____ ville jobbe som [OCCUPATION].",
    "____ kommer til å jobbe som [OCCUPATION].",
    "____ begynte å jobbe som [OCCUPATION].",
    "____ har jobbet som [OCCUPATION].",
    "____ hadde jobbet som [OCCUPATION].",
    "____ kunne jobbe som [OCCUPATION].",
    "____ ønsker å jobbe som [OCCUPATION].",
    "____ pleide å jobbe som [OCCUPATION].",
    "____ fortsetter å jobbe som [OCCUPATION].",
    "____ drømmer om å jobbe som [OCCUPATION].",
    "____ skal snart jobbe som [OCCUPATION].",
    "____ fikk en jobb som [OCCUPATION].",
    "____ søkte på en jobb som [OCCUPATION].",
    "____ planlegger å jobbe som [OCCUPATION].",
    "____ kan jobbe som [OCCUPATION].",
    "____ lærte å jobbe som [OCCUPATION]."
]

# gender distribution across occupations in norway 2023. (utdanning.no)
gold_data = readCSV("../censusData/preparedData/utdanningnoLikestilling2023_ppbs.csv")
occs = list(map(lambda x: x[0],gold_data))
bert = "[MASK]"
roberta = "<mask>"

In [4]:
##########################################################################
# probing gender given occupation
pred_data = probePPBS(occs, templates, roberta, "FacebookAI/xlm-roberta-base")
writeCSV("data/raw/xlmRBase_ppbs.csv", gold_data, pred_data)

Some weights of the model checkpoint at FacebookAI/xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Occupation: 100%|██████████| 450/450 [11:45<00:00,  1.57s/it]


In [5]:
pred_data = probePPBS(occs, templates, roberta, "FacebookAI/xlm-roberta-large")
writeCSV("data/raw/xlmRLarge_ppbs.csv", gold_data, pred_data)

Some weights of the model checkpoint at FacebookAI/xlm-roberta-large were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Occupation: 100%|██████████| 450/450 [21:15<00:00,  2.84s/it]


In [6]:
pred_data = probePPBS(occs, templates, bert, "google-bert/bert-base-multilingual-uncased")
writeCSV("data/raw/mBertUncased_ppbs.csv", gold_data, pred_data)

Some weights of the model checkpoint at google-bert/bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Occupation: 100%|██████████| 450/450 [06:28<00:00,  1.16it/s]


In [7]:
pred_data = probePPBS(occs, templates, bert, "google-bert/bert-base-multilingual-cased")
writeCSV("data/raw/mBertCased_ppbs.csv", gold_data, pred_data)

Some weights of the model checkpoint at google-bert/bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Occupation: 100%|██████████| 450/450 [08:15<00:00,  1.10s/it]


In [8]:
pred_data = probePPBS(occs, templates, bert, "NbAiLab/nb-bert-base")
writeCSV("data/raw/nbBertBase_ppbs.csv", gold_data, pred_data)

Occupation: 100%|██████████| 450/450 [08:03<00:00,  1.07s/it]


In [9]:
pred_data = probePPBS(occs, templates, bert, "NbAiLab/nb-bert-large")
writeCSV("data/raw/nbBertLarge_ppbs.csv", gold_data, pred_data)

Occupation: 100%|██████████| 450/450 [10:10<00:00,  1.36s/it]


In [10]:
pred_data = probePPBS(occs, templates, bert, "ltg/norbert")
writeCSV("data/raw/norbert_ppbs.csv", gold_data, pred_data)

Some weights of the model checkpoint at ltg/norbert were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Occupation: 100%|██████████| 450/450 [04:06<00:00,  1.82it/s]


In [11]:
pred_data = probePPBS(occs, templates, bert, "ltg/norbert2")
writeCSV("data/raw/norbert2_ppbs.csv", gold_data, pred_data)

Some weights of the model checkpoint at ltg/norbert2 were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Occupation: 100%|██████████| 450/450 [04:18<00:00,  1.74it/s]
